In [1]:
import os
import glob
import numpy as np
import pandas as pd
from functions import metrics
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
os.chdir("/Users/pestephan/Documents/GitHub/Machine-Learning-using-Sports-Data/Predicting-PremLeague-Outcomes-With-ML/files/Datasets")

Data Taken from http://www.football-data.co.uk/englandm.php

In [2]:
extension = 'csv'
all_filenames = [i for i in glob.glob('*.{}'.format(extension))]

In [3]:
#combine all files in the list
master_sheet = pd.concat([pd.read_csv(f) for f in all_filenames ])
#export to csv
master_sheet.to_csv( "master_sheet.csv", index=False, encoding='utf-8-sig')

Reduce table to using columns needed for the model

Columns Required:

Date, Time HomeTeam, AwayTeam, FullTimeHG, FullTimeAG, FullTimeRresults, HalfTimeHG, HalfTimeAG, HomeShots, AwayShots, HomeShotsTarget, AwayShotsTarget, HomeCorners, AwayCorners, HomeFouls, AwayFouls, HomeRed, AwayRed

Columns To Consider:

Attendance, Referee

In [4]:
#"HTHG", "HTAG", 
subset_master = master_sheet[["Date", "Time", "HomeTeam", "AwayTeam", "FTHG", "FTAG", "FTR", "HS", "AS", "HST", "AST", "HC", "AC", "HF", "AF", "HR", "AR"]]
subset_master = subset_master.fillna(0)
# 0 = Home Wins, 1 = Draw, 2 = Away Wins
subset_master.FTR.replace(['H', 'D', 'A'], [0, 1, 2], inplace = True)

In [5]:
subset_master.head()

,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HS,AS,HST,AST,HC,AC,HF,AF,HR,AR
0,13/08/11,0,Blackburn,Wolves,1.0,2.0,2,16.0,13.0,8.0,4.0,12.0,6.0,14.0,10.0,0.0,0.0
1,13/08/11,0,Fulham,Aston Villa,0.0,0.0,1,13.0,7.0,9.0,1.0,2.0,3.0,10.0,18.0,0.0,0.0
2,13/08/11,0,Liverpool,Sunderland,1.0,1.0,1,11.0,15.0,4.0,6.0,6.0,3.0,17.0,12.0,0.0,0.0
3,13/08/11,0,Newcastle,Arsenal,0.0,0.0,1,6.0,9.0,1.0,4.0,2.0,5.0,9.0,11.0,0.0,1.0
4,13/08/11,0,QPR,Bolton,0.0,4.0,2,13.0,13.0,7.0,7.0,3.0,2.0,9.0,16.0,1.0,0.0


In [6]:
msk = np.random.rand(len(subset_master)) < 0.70

train_df = subset_master[msk]
test_df = subset_master[~msk]

X_train = train_df.drop(columns = ['Date', 'Time', 'HomeTeam', 'AwayTeam', 'FTR'])
y_train = train_df[['FTR']] 
X_test = test_df.drop(columns = ['Date', 'Time', 'HomeTeam', 'AwayTeam', 'FTR'])
y_test = test_df[['FTR']]

In [7]:
clf = MLPClassifier(activation = 'logistic',solver='sgd', alpha=0.001, learning_rate = 'adaptive', random_state=1)
clf.fit(X_train, y_train)

y_pred = clf.predict_proba(X_test)
y_pred = pd.DataFrame(y_pred)

Identifying how often Predicted Results decided the outcome (win, draw, away) against the actual results)

In [8]:
metrics(subset_master, y_pred)

False    64.067227
True     35.932773
Name: Comparison, dtype: float64
0    45.394958
2    30.240096
1    24.364946
Name: FTR, dtype: float64
0    45.260504
2    29.858343
1    24.881152
Name: PredResult, dtype: float64


In [9]:
subset_master

,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HS,AS,HST,AST,HC,AC,HF,AF,HR,AR,HomeWinPerc,DrawPerc,AwayPerc
0,13/08/11,0,Blackburn,Wolves,1.0,2.0,2,16.0,13.0,8.0,4.0,12.0,6.0,14.0,10.0,0.0,0.0,8.487173e-03,9.884380e-01,3.074877e-03
1,13/08/11,0,Fulham,Aston Villa,0.0,0.0,1,13.0,7.0,9.0,1.0,2.0,3.0,10.0,18.0,0.0,0.0,4.376005e-03,9.880764e-01,7.547594e-03
2,13/08/11,0,Liverpool,Sunderland,1.0,1.0,1,11.0,15.0,4.0,6.0,6.0,3.0,17.0,12.0,0.0,0.0,7.024793e-18,2.770142e-07,9.999997e-01
3,13/08/11,0,Newcastle,Arsenal,0.0,0.0,1,6.0,9.0,1.0,4.0,2.0,5.0,9.0,11.0,0.0,1.0,4.072037e-03,9.891587e-01,6.769235e-03
4,13/08/11,0,QPR,Bolton,0.0,4.0,2,13.0,13.0,7.0,7.0,3.0,2.0,9.0,16.0,1.0,0.0,9.999999e-01,9.370249e-08,8.937673e-19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
376,24/05/15,0,Leicester,QPR,5.0,1.0,0,22.0,18.0,7.0,2.0,5.0,6.0,7.0,6.0,0.0,0.0,9.999999e-01,9.972869e-08,8.840234e-19
377,24/05/15,0,Man City,Southampton,2.0,0.0,0,15.0,13.0,6.0,4.0,8.0,4.0,13.0,8.0,0.0,0.0,8.467299e-03,9.861863e-01,5.346434e-03
378,24/05/15,0,Newcastle,West Ham,2.0,0.0,0,17.0,4.0,4.0,1.0,2.0,3.0,9.0,9.0,0.0,0.0,9.999995e-01,4.597779e-07,1.860638e-17
379,24/05/15,0,Stoke,Liverpool,6.0,1.0,0,15.0,13.0,9.0,4.0,3.0,9.0,13.0,4.0,0.0,0.0,3.683527e-09,8.249347e-03,9.917506e-01
